# Learning from Inflation Experiences
In this notebook, I demonstrate my recreation of the main analysis of [Malmendier & Nagel (2016)](https://academic.oup.com/qje/article-abstract/131/1/53/2461168). They regress individuals' reported inflation expectations on theoretical expectations calculated from experienced inflation using a recursive learning algorithm.

In [20]:
path = 'D:\Studium\Master\Masterarbeit\Memory-Expectations'

# import packages
import pandas as pd
import numpy as np

# Importing some selected functions from packages
import statsmodels.formula.api as smf

First, import the 
- realized inflation time series
- survey data reporting individuals' actual inflation expectations

In [25]:
malnag_data_infl     = pd.read_excel(path + r'\data\infl.xlsx')
malnag_data_survey   = pd.read_excel(path + r'\data\cohfile.xlsx')

Next, we need to define some parameters for the recursive learning algorithm (RLA):

In [26]:
parameters = {}

parameters["year_start"]    = 1872
parameters["year_end"]      = 2009
parameters["cohort_start"]  = 1872
parameters["cohort_end"]    = 1984
parameters["maxage"]        = 76


parameters["s_range"]       = np.arange(parameters["year_start"], parameters["cohort_end"] + 1)
parameters["t_range"]       = np.arange(parameters["year_start"], parameters["year_end"] + 1)

In [27]:
malnag_data_infl_array = np.array(malnag_data_infl.loc[
    (malnag_data_infl['yyyyqtr'] >= parameters["year_start"]*100+1) &
    (malnag_data_infl['yyyyqtr'] <= parameters["year_end"]*100+4)]
    ["yrinfl"])

The RLA further takes a matrix of gain parameters (gamma) as input. These values define, how much a new observation of inflation updates an individual's expectations.

In [23]:
def MalNagGammaMatrixQuarterly(t_range,s_range,theta):
    gamma = np.zeros((len(t_range)*4, len(s_range))) # creating an empty gamma matrix
    gamma = np.where(gamma == 0, np.nan, gamma) # filling it with nan
    tmin = min(t_range) # minimum values are necessary to adjust row and column indices
    smin = min(s_range)
    for t in t_range: # loop over all observed years
        for q in range(1,4+1): # loop over all quarters is said years
            for s in s_range: # loop over all birth cohorts
                ageqtr = (t-s)*4+q-1 # calculate age in quarters
                # This is the gain function, that M&N2016 rely on
                if ageqtr >= 2*theta:
                    g = theta/ageqtr
                else:
                    g = 0.5
                rowindex = (t-tmin)*4+q-1
                colindex = s-smin
                gamma[rowindex,colindex] = g # write calculated gamm into matrix
    return gamma

M&N2016 find a gain a $\theta$ = 3.044 to be optimal; use it to cretae the gamma matrix:

In [24]:
theta = 3.044
gamma = MalNagGammaMatrixQuarterly(parameters["t_range"],
                                    parameters["s_range"],
                                    theta)

Next, we need do define the recursive learning algorithm:

In [28]:
def MalNagRLAQuarterly(t_range, s_range, data_start, maxage, inflation_array, gamma_array, initperiod):
    # create empty expectations array
    expectations = np.zeros((len(t_range)*4, len(s_range)))
    expectations = np.where(expectations == 0, np.nan, expectations)
    
    # again, minimum values are necessary for row and column indices
    tmin = min(t_range)
    smin = min(s_range)

    for s in s_range: # loop over all birth cohorts
        for t in t_range: # loop over all observed years
            for q in range(1,4+1): # loop over all quarters

                rowindex    = (t-tmin)*4+q-1
                colindex    = s-smin
                
                # get this quarter's relaized inflation from time series
                infl        = inflation_array[rowindex]

                # during the initialization period, parameters of RLA remain unchanged
                if (t < data_start + initperiod):
                    R       = np.array([[1,0],[0,0]])
                    phit    = infl
                    cc      = np.array([phit,0])
                    x       = np.array([1,0])
                    c       = cc
                
                # after the initialization period, with each quarter of new realized inflation,
                # parameter estimates are updated, and individual forms new expectations
                # based on updated parameters and new information
                elif (t < s + maxage):
                    infl_l1 = float(inflation_array[rowindex-1]) # last period's realized inflation
                    x       = np.array([1,infl_l1])
                    y       = infl
                    perr    = y     - np.matmul(cc,x) # last period's prediction error
                    
                    # This is where the 'learning' takes place,
                    # gain parameter gamma controls how much parameters R and cc are updated
                    R       = R     + gamma_array[rowindex,colindex]*(np.outer(x,x)-R)
                    cc      = cc    + gamma_array[rowindex,colindex]*(np.matmul(np.linalg.pinv(R),x))*perr
                    c       = cc
                    xcurr   = np.array([1,infl])
                    phit    = np.matmul(c,xcurr) # this is the new inflation expectation
                    xf      = np.array([1,phit])
                    
                    # adjust expected inflation for quarterly rythm
                    for h in range(1,4,1):
                        pf = np.matmul(c,xf)
                        phit = phit + pf
                        xf = np.array([1,pf])
                    phit = phit/(3+1)

                expectations[rowindex,colindex] = phit # write expectation into matrix

    return expectations

Given the RLA parameters defined above, a gamma matrix and an initialization period of 20 years, we can simulate the RLA:

In [30]:
expectations = MalNagRLAQuarterly(t_range            = parameters["t_range"],
                                         s_range            = parameters["s_range"],
                                         data_start         = parameters["year_start"],
                                         maxage             = parameters["maxage"],
                                         inflation_array    = malnag_data_infl_array,
                                         gamma_array        = gamma,
                                         initperiod         = 20)

Before we can run the regression, we need to match inflation expectations to the survey. The matching is based on
- year of birth (columns)
- quarter of observation (rows)

In [32]:
def MalNagMatchExpToSurvey(survey, expectations, parameters):
    survey['ir'] = (survey['yyyy'] - parameters["year_start"]) * 4 + survey['q'] - 1
    survey['ic'] =  survey['yyyy'] - survey['age'] - (parameters["year_start"] - 1)
    survey['xnl'] = 0

    for index, row in survey.iterrows():
        ir = survey.loc[int(index),'ir']-1
        ic = survey.loc[int(index),'ic']-1
        survey.loc[int(index),'xnl'] = expectations[ir,ic]

    return survey

regsample = MalNagMatchExpToSurvey(survey       = malnag_data_survey,
                                   expectations = expectations,
                                   parameters   = parameters)
regsample.head()

,yyyy,age,q,yyyyq,cohort,infl1pct,ir,ic,xnl
0,1953,30,4,19534,1923,-0.003495,327,52,0.031333
1,1953,40,4,19534,1913,-0.011470,327,42,0.026422
2,1953,50,4,19534,1903,-0.014711,327,32,0.024281
3,1953,60,4,19534,1893,-0.004793,327,22,0.023466
4,1953,70,4,19534,1883,-0.008608,327,12,0.022929


Lastly, we run the M&N2016 main regression:

In [33]:
model_1 = smf.ols('infl1pct ~ xnl + C(yyyyq) - 1',data = regsample)
model_1 = model_1.fit(cov_type="cluster",
                  cov_kwds={"groups": [regsample['yyyyq'],regsample['cohort']]})

print("M&N2016 have a beta = 0.672 (0.076), we find beta = "
      + str(round(model_1.params[-1],4)) + " (" + str(round(model_1.bse[-1], 4)) + ")")

M&N2016 have a beta = 0.672 (0.076), we find beta = 0.6715 (0.0733)
